# Geocentric Pose

**Authors:** Christopher Sun, Jai Sharma, Milind Maiti

**Date:** 2022.06.16

**Description:** This module concerns the task of predicting the Geocentric Pose of a satellite given the j2k RGB image taken from that satellite. The tasks here include:

1. Import the j2k images, pixelwise elevation tifs (which have been interpolated), and the scale and angle labels.
2. Build and train the Geocentric Pose Deep Learning model.
3. Vizualize learning curves for the model trained in number (2). 
4. Find the $R^2$ score for the scale and angle predictions.

## Import Libraries and Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.pyplot import figure
from mpl_toolkits import mplot3d
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, Sequential, Input, Model
from tensorflow.keras.layers import *
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import math
import scipy
import pickle
import gc 
from IPython.display import display

# Print Confirmation
print("Setup Complete")

In [2]:
# Import Data
j2ks = pickle.load(open("/kaggle/input/geocentric-pose-analysis-of-satellite-imagery/j2k_imgs.dat", "rb"))
tifs = pickle.load(open("/kaggle/input/geocentric-pose-analysis-of-satellite-imagery/new_tifs.dat", "rb"))
y = pickle.load(open("/kaggle/input/geocentric-pose-analysis-of-satellite-imagery/y.dat", "rb"))

# Normalize Scale Values
y[:,0] = y[:,0] * 1e3

# X values
X = j2ks

## Define and Train Geocentric Pose Model

In [ ]:
# Geocentric Pose model
def build_model():
    inputs = Input(shape=(256,256,3))
    initializer = tf.keras.initializers.GlorotNormal(seed=1)
    x = Conv2D(32, (5,5), padding="same", kernel_initializer=initializer)(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(32, (5,5), padding="same", kernel_initializer=initializer)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D((3,3), strides=2)(x)
    
    x = Conv2D(64, (5,5), padding="same", kernel_initializer=initializer)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)   
    
    x = Conv2D(64, (5,5), padding="same", kernel_initializer=initializer)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)  
    x = MaxPooling2D((3,3), strides=2)(x)
  
    x = Conv2D(128, (5,5), padding="same", kernel_initializer=initializer)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(128, (5,5), padding="same", kernel_initializer=initializer)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x) 
    x = MaxPooling2D((3,3), strides=2)(x)
   
    x = Conv2D(256, (5,5), padding="same", kernel_initializer=initializer)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D((3,3), strides=2)(x)
    
    x = Conv2D(256, (5,5), padding="same", kernel_initializer=initializer)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D((3,3), strides=2)(x)

    x = Flatten()(x)
    x = Dense(300)(x)
    x = Activation("relu")(x)

    x = Dense(150)(x)
    x = Activation("relu")(x)
    
    x = Dense(50)(x)
    x = Activation("relu")(x)
    
    x = Dense(16)(x)
    x = Activation("relu")(x)
    
    x = Dense(8)(x)
    x = Activation("relu")(x)
    x = Dense(2)(x)
    
    outputs = Activation("relu")(x)
    model = Model(inputs, outputs)
    
    return model

In [ ]:
# Build and Compile the model
model = build_model() 
model.compile(optimizer="adam", loss="mae", metrics=["mse"])

In [ ]:
# View the model architecture summary
model.summary()

In [ ]:
# Split data into train and validation sets
X_t, X_v, y_t, y_v = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# Train Geocentric Pose model
model.fit(X_t, y_t, validation_data=(X_v, y_v), epochs=500)

## Calculate Metrics

Here are the results for the U-Net model:

**Train $R^2$ for scale:** 0.9903

**Train $R^2$ for angle:** 0.9910

**Validation $R^2$ for scale:**   0.9238

**Validation $R^2$ for angle:**   0.9625

In [ ]:
# Find R^2 metrics for Scale and Angle
train_pred = model.predict(X_t)
pred = model.predict(X_v)

def r_squared_scale(true, pred):
    R2 = 1 - np.sum((true[:,0] - pred[:,0])**2)/np.sum((true[:,0] - np.mean(true[:,0]))**2)
    
    return R2

def r_squared_angle(true, pred):
    lst = []
    for i in range(pred[:,1].shape[0]):
        error = np.abs(pred[i,1] - true[i,1])
        if 6.28 - error < error:
            lst.append(6.28 - error)
        else:
            lst.append(error)
    
    R2 = 1 - np.sum((np.array(lst))**2)/np.sum((true[:,1] - np.mean(true[:,1]))**2)
    
    return R2

In [ ]:
print("R^2 for scale for train: " + str(r_squared_scale(y_t, train_pred)))
print("R^2 for angle for train: " + str(r_squared_angle(y_t, train_pred)))
print("R^2 for scale for validation: " + str(r_squared_scale(y_v, pred)))
print("R^2 for angle for validation: " + str(r_squared_angle(y_v, pred)))